In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
from datetime import date

In [2]:
#SET DATE FILTERS & FileName
today = date.today()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('-%m%d%Y')

file = "Int_MAM"+yesterday+".csv"
start = datetime.today()- timedelta(days=1)
start =start.strftime('%Y-%m-%d')

end = datetime.strptime(start, "%Y-%m-%d") - timedelta(days=6)
end = end.strftime('%Y-%m-%d')

start14 = datetime.strptime(start,"%Y-%m-%d") - timedelta(days=13)
start14 = start14.strftime('%Y-%m-%d')

startPW = datetime.strptime(start,"%Y-%m-%d") - timedelta(days=7)
startPW = startPW.strftime('%Y-%m-%d')

print(start)
print(end)
print(startPW)
print(start14)

2020-10-15
2020-10-09
2020-10-08
2020-10-02


In [3]:
int_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

In [4]:
#Cases per 100k
int_df['new_cases_100k'] = ((int_df['new_cases']/int_df['population'])*100000)
int_df['prior_new_cases_100k'] = ((int_df['new_cases']/int_df['population'])*100000)
int_df['active_cases_100k'] = ((int_df['new_cases']/int_df['population'])*100000)

In [5]:
#Filter data set - 7day avg new cases
range_7 = int_df[(int_df['date']<=start) & (int_df['date']>=end)]
case_grouped_mean = range_7.groupby(['location'], as_index=False).mean()
base = pd.DataFrame(case_grouped_mean[['location','new_cases_100k']])

In [6]:
#Filter data set - active cases
range_14 = int_df[(int_df['date']>=start14) & (int_df['date']<=start)]
case_grouped_sum = range_14.groupby(['location'], as_index=False).sum()

In [7]:
#Filter data set - prior week cases
prior= int_df[(int_df['date']>=start14) & (int_df['date']<=startPW)]
prior_grouped = prior.groupby(['location'], as_index=False).mean()

In [8]:
int_mam1 = pd.merge(base,
                 case_grouped_sum[['location','active_cases_100k']],
                 on='location',
                 how='left')

In [9]:
int_mam2 = pd.merge(int_mam1,
                 prior_grouped[['location','prior_new_cases_100k']],
                 on='location',
                 how='left')

int_mam2['wow_case_index'] = ((int_mam2['new_cases_100k']-int_mam2['prior_new_cases_100k'])/int_mam2['prior_new_cases_100k'])
print(int_mam2.head())

      location  new_cases_100k  active_cases_100k  prior_new_cases_100k  \
0  Afghanistan        0.150460           1.947167              0.127707   
1      Albania        6.081034          80.130655              5.366202   
2      Algeria        0.346628           4.684039              0.322520   
3      Andorra      115.003097        1475.441662             95.774283   
4       Angola        0.415102           5.196818              0.327300   

   wow_case_index  
0        0.178161  
1        0.133210  
2        0.074747  
3        0.200772  
4        0.268260  


In [10]:
#Set ranges

def curr7_hrvd(n):
    if -222<=n< 2: score = 0
    elif 2 <=n<10: score =1
    elif 10 <=n<25: score =2
    elif 25 <=n<=99999: score =3
    else: score = 0
    return(score)

def roll_hrvd(n):
    if -10000<=n<0.10: score = 0
    elif 0.10<=n<=10000: score = 0.5  
    else: score = 0
    return(score)

def county_hrvd(n):
    if 0<=n<0.5: score = "Minimal"
    elif 0.5<=n<1: score = "Low"
    elif 1<=n<2: score = "Moderate"
    elif 2<=n<3: score = "Elevated"
    elif 3<=n<= 100: score = "Critical"
    else: score = 0
    return(score)

In [11]:
int_mam2['new_case_score'] = int_mam2['new_cases_100k'].apply(curr7_hrvd)
int_mam2['wow_score'] = int_mam2['wow_case_index'].apply(roll_hrvd)

int_mam2['country_composite'] = (int_mam2['new_case_score']+int_mam2['wow_score'])

int_mam2['Country_Classification'] = int_mam2['country_composite'].apply(county_hrvd)

final_MAM = pd.DataFrame(int_mam2[['location','Country_Classification','new_cases_100k','wow_case_index','active_cases_100k']])
print(final_MAM.head())
final_MAM.to_csv("Final_Int_mam.csv")

      location Country_Classification  new_cases_100k  wow_case_index  \
0  Afghanistan                    Low        0.150460        0.178161   
1      Albania               Moderate        6.081034        0.133210   
2      Algeria                Minimal        0.346628        0.074747   
3      Andorra               Critical      115.003097        0.200772   
4       Angola                    Low        0.415102        0.268260   

   active_cases_100k  
0           1.947167  
1          80.130655  
2           4.684039  
3        1475.441662  
4           5.196818  


In [12]:
# #Filter for Countries with our presence
# Int_countries_filter = ['United States','Spain','Singapore','India','Netherlands','France','Ireland','United Kingdom',
#                        'Germany','Canada','Italy','China']

# final_MAM_filtered = final_MAM[final_MAM.location.isin(Int_countries_filter)]

In [13]:
# #Spit out CSV
# final_MAM_filtered.to_csv(file, index=False) 
